In [ ]:
"../human/clapnq_questions.jsonl"
"../human/clapnq_rewrite.jsonl"
"../human/retrieval_tasks_convid/cloud/qrels/dev.tsv"

In [2]:
import json

def load_queries(jsonl_path):
    queries = {}
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            qid = obj.get("_id")
            text = obj.get("text")
            queries[qid] = text
    return queries

CLAPNQ_QUERY_PATH = "../human/clapnq_questions.jsonl"   # add correct path
queries = load_queries(CLAPNQ_QUERY_PATH)
print(f"Loaded {len(queries)} queries")


Loaded 208 queries


In [ ]:
from pymilvus import Collection

def retrieve_top_k(query_texts, top_k=100):
    results = {}
    for qid, text in query_texts.items():
        q_emb = model.encode([text], convert_to_numpy=True)

        search_params = {"metric_type": "COSINE", "params": {"nprobe": 16}}

        raw = collection.search(
            data=q_emb,
            anns_field="embedding",
            param=search_params,
            limit=top_k,
            output_fields=["text"]
        )[0]

        results[qid] = [
            (hit.id, float(hit.score)) for hit in raw
        ]
    return results


In [ ]:
def create_evaluation_format(results_with_scores, collection_name="clapnq_passages"):
    output = []
    for qid, docs in results_with_scores.items():
        contexts = []
        for doc_id, sim_score in docs:
            contexts.append({
                "document_id": doc_id,
                "score": sim_score
            })
        output.append({
            "task_id": qid,
            "Collection": collection_name,
            "contexts": contexts
        })
    return output


In [ ]:
def save_predictions(evaluation_data, filename):
    with open(filename, "w") as f:
        for item in evaluation_data:
            f.write(json.dumps(item) + "\n")

save_predictions(evaluation_data, "clapnq_retrieval_predictions.jsonl")


In [ ]:
import subprocess
eval_script = "../scripts/evaluation/run_retrieval_eval.py"

subprocess.run([
    "python",
    eval_script,
    "--input_file", "clapnq_retrieval_predictions.jsonl",
    "--output_file", "clapnq_retrieval_eval.jsonl"
])
